In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers 
import re 
# re — Regular expression operations
import math
import csv
import pandas as pd                     
import cv2 as cv 
from tensorflow.keras import losses
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from google.colab import drive
# os — Miscellaneous operating system interfaces
import os 
from PIL import Image, ImageSequence

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
r_data = pd.read_csv("/content/drive/Shareddrives/Animated Gif/GIF models/redditgif/reddit_data_brightness")

In [ ]:
r_data.head()

Unnamed: 0  Unnamed: 0.1  ...   filenames brightness
0           0             1  ...  mt0nqt.gif   0.173433
1           1             2  ...  msz066.gif   0.119948
2           2             3  ...  msyt6a.gif   0.217824
3           3             4  ...  msy15j.gif   0.372015
4           4             5  ...  mssnzt.gif   0.316349

[5 rows x 8 columns]

In [ ]:
os.chdir('/content/drive/Shareddrives/Animated Gif/GIF models/redditgif')

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def calculate_image_brightness(image):
    greyscale_image = image.convert('L')
    histogram = greyscale_image.histogram()
    pixels = sum(histogram)
    brightness = scale = len(histogram)
    for index in range(0, scale):
        ratio = histogram[index] / pixels
        brightness += ratio * (-scale + index)
    return 1 if brightness == 255 else brightness / scale
def calculate_gif_brightness(path_to_gif_file):
    capture = Image.open(path_to_gif_file)
    gif_brightness = []
    for frame in ImageSequence.Iterator(capture):
        gif_brightness.append(calculate_image_brightness(frame))
    return sum(gif_brightness) / len(gif_brightness)

In [20]:
def get_avg_contrast(path_to_gif):
    gif_contrast = []
    capture = cv.VideoCapture(path_to_gif)
    pos_frame = capture.get(cv.CAP_PROP_POS_FRAMES)

    while True: 
        isTrue, frame = capture.read()
        img_grey = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        gif_contrast.append(img_grey.std())
        if capture.get(cv.CAP_PROP_POS_FRAMES) == capture.get(cv.CAP_PROP_FRAME_COUNT):
            # If the number of captured frames is equal to the total number of frames,
            # we stop
            break
    return math.log(sum(gif_contrast) / len(gif_contrast))

In [14]:
def get_number_of_pixels(path_to_gif):
    capture = cv.VideoCapture(path_to_gif)
    pos_frame = capture.get(cv.CAP_PROP_POS_FRAMES)

    # Capture first frame 
    ret, frame = capture.read()
    first_frame_size = frame.shape
  
    return first_frame_size[0] * first_frame_size[1]

In [ ]:
brightness = []
i = 0
for gif_file_path in r_data['filenames']: 
    #Image.open(gif_file_path)
    brightness.append(calculate_gif_brightness(gif_file_path))
    i = i+1
    print(str(i)+" "+gif_file_path)

In [ ]:
brightness
r_data['brightness'] = brightness


In [ ]:
r_data.to_csv("/content/drive/Shareddrives/Animated Gif/GIF models/reddit_data_brightness")

In [17]:
get_number_of_pixels("2r8x12.gif") / 100000

0.35

In [ ]:
for gif_file_path in r_data['filenames'][:5]: 
    print(get_number_of_pixels(gif_file_path) / 100000)

0.59987
3.072
1.025
0.484
0.375


In [22]:
get_avg_contrast("2r91p9.gif")

3.854267725831638